In [ ]:
# orchestrator.py

from langgraph.graph import StateGraph, MessagesState, START, END

# Import agent functions
from agents.coordinator import coordinator

# Import Retriever agent functions
from agents.retrievers.bm25_agent import bm25_agent
from agents.retrievers.e5_agent import e5_agent
from agents.retrievers.specter_agent import specter_agent

# from agents.llm_agent import llm_agent
from agents.formulators.query_reformulator import query_reformulator
from agents.formulators.analysis_agent import analysis_agent
from agents.formulators.reranker import reranker
# from agents.verifier_agent import verifier_agent


############################################################
# 1️⃣  BUILD THE STATE GRAPH (use MessagesState!)
############################################################

# IMPORTANT: Studio REQUIRES MessagesState or Annotated fields
graph = StateGraph(MessagesState)

############################################################
# 2️⃣  ADD AGENTS (nodes)
############################################################

graph.add_node("reformulator", query_reformulator)
# graph.add_node("coordinator", coordinator)
graph.add_node("bm25", bm25_agent)
graph.add_node("e5", e5_agent)
graph.add_node("specter", specter_agent)
# graph.add_node("llm", llm_agent, tags=["retriever"])
graph.add_node("analysis", analysis_agent)  # Placeholder analysis agent
# graph.add_node("verifier", verifier_agent, tags=["agent"])
graph.add_node("reranking", reranker)
############################################################
# 3️⃣  ADD EDGES
############################################################

# Start → Query Reformulator -> Coordinator
graph.add_edge(START, "reformulator")
# graph.add_edge("reformulator", "coordinator")

# Coordinator fans out to all retrieval agents
graph.add_edge("reformulator", "bm25")
graph.add_edge("reformulator", "e5")
graph.add_edge("reformulator", "specter")
# graph.add_edge("coordinator", "llm")

# All retrieval agents → analysis agent
graph.add_edge("bm25", "analysis")
graph.add_edge("e5", "analysis")
graph.add_edge("specter", "analysis")
# graph.add_edge("llm", "analysis")

# Analysis -> Verifier agent -> END
graph.add_edge("analysis", "reranking")
graph.add_edge("reranking", END)
# graph.add_edge("verifier", END)

############################################################
# 4️⃣ EXPOSE PIPELINE FOR LANGGRAPH STUDIO
############################################################

pipeline = graph.compile()

In [ ]:
from IPython.display import Image

graph_image = pipeline.get_graph().draw_mermaid_png()
Image(graph_image)